In [45]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams
import string
import unicodedata
import re
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

!pip install fuzzywuzzy

# Baixar stopwords (caso ainda não tenha)
import nltk
nltk.download('stopwords')

# Exemplo de dados de ofertas
ofertas = [
    "Oferta especial na compra de um produto",
    "Desconto imperdível em diversos itens",
    "Promoção exclusiva para clientes VIP",
    "Aproveite os preços baixos desta semana"
]

# Função para preprocessamento avançado dos bigramas
def preprocessamento_avancado(oferta):
    # Tokenização e conversão para minúsculas
    tokens = word_tokenize(oferta.lower())

    # Remoção de stopwords e pontuações
    stop_words = set(stopwords.words('portuguese'))
    tokens = [token for token in tokens if token not in stop_words and token not in string.punctuation]

    # Stemming (redução à forma básica)
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]

    # Remoção de palavras curtas
    tokens = [token for token in tokens if len(token) > 2]

    # Remoção de acentuação
    tokens = [unicodedata.normalize('NFKD', token).encode('ASCII', 'ignore').decode('utf-8') for token in tokens]

    # Remoção de números e caracteres especiais
    tokens = [re.sub(r'[^a-zA-Z]', '', token) for token in tokens]

    # Lematização (convertendo palavras para sua forma base)
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Reconstrução dos bigramas
    bigrams = list(ngrams(tokens, 2))

    return bigrams

# Criar uma lista de bigramas preprocessados para cada oferta
lista_de_bigramas_preprocessados = [preprocessamento_avancado(oferta) for oferta in ofertas]

# Criar um DataFrame com os bigramas preprocessados e sem pontuações/acentuação
df = pd.DataFrame(data={"Oferta": ofertas, "Bigramas Preprocessados": lista_de_bigramas_preprocessados})

# Mostrar o DataFrame
print(df)

# Salvar o DataFrame em um arquivo CSV
df.to_csv('bigram_data.csv', index=False)

# Salvar a lista de bigramas em um arquivo JSON
bigram_dict_list = [{"bigramas_venda": [{"bigrama": bigrama} for bigrama in bigramas]} for bigramas in lista_de_bigramas_preprocessados]
with open('bigram_file.json', 'w') as json_file:
    json.dump(bigram_dict_list, json_file, ensure_ascii=False, indent=2)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                    Oferta  \
0  Oferta especial na compra de um produto   
1    Desconto imperdível em diversos itens   
2     Promoção exclusiva para clientes VIP   
3  Aproveite os preços baixos desta semana   

                             Bigramas Preprocessados  
0  [(oferta, especi), (especi, compra), (compra, ...  
1  [(desconto, imperdivel), (imperdivel, diverso)...  
2  [(promocao, exclusiva), (exclusiva, client), (...  
3  [(aproveit, preco), (preco, baixo), (baixo, de...  


In [46]:
# Exemplo de dados de ofertas
ofertas2 = [
    "Super desconto na aquisição de qualquer item",
    "Oferta irresistível em uma variedade de produtos",
    "Promoção exclusiva para membros VIP",
    "Não perca as grandes ofertas desta semana"
]


In [47]:
# Criar uma lista de bigramas preprocessados para cada oferta
lista_de_bigramas_preprocessados = [preprocessamento_avancado(oferta) for oferta in ofertas2]

# Criar um DataFrame com os bigramas preprocessados e sem pontuações/acentuação
df3 = pd.DataFrame(data={"Oferta": ofertas2, "Bigramas Preprocessados": lista_de_bigramas_preprocessados})

# Mostrar o DataFrame
print(df3)

# Salvar o DataFrame em um arquivo CSV
df3.to_csv('bigram_data.csv', index=False)

# Salvar a lista de bigramas em um arquivo JSON
bigram_dict_list = [{"bigramas_venda": [{"bigrama": bigrama} for bigrama in bigramas]} for bigramas in lista_de_bigramas_preprocessados]
with open('bigram_file.json', 'w') as json_file:
    json.dump(bigram_dict_list, json_file, ensure_ascii=False, indent=2)

                                             Oferta  \
0      Super desconto na aquisição de qualquer item   
1  Oferta irresistível em uma variedade de produtos   
2               Promoção exclusiva para membros VIP   
3         Não perca as grandes ofertas desta semana   

                             Bigramas Preprocessados  
0  [(super, desconto), (desconto, aquisicao), (aq...  
1  [(oferta, irresistivel), (irresistivel, varied...  
2  [(promocao, exclusiva), (exclusiva, membro), (...  
3  [(perca, grand), (grand, oferta), (oferta, des...  


In [48]:
import pandas as pd
from fuzzywuzzy import fuzz
from typing import List

class BigramaMatcher:
    def __init__(self, df1: pd.DataFrame, df2: pd.DataFrame, column_name_df1: str = 'Bigramas Preprocessados', column_name_df2: str = 'Bigramas Preprocessados', fuzz_threshold: int = 70) -> None:
        self.df1 = df1
        self.df2 = df2
        self.column_name_df1 = column_name_df1
        self.column_name_df2 = column_name_df2
        self.fuzz_threshold = fuzz_threshold
        self.result_df = pd.DataFrame()

    @staticmethod
    def format_bigrams_as_tuples(df: pd.DataFrame, column: str) -> pd.DataFrame:
        df[column] = df[column].apply(lambda x: [tuple(bigram) for bigram in x])
        return df

    def check_bigram_presence(self, row1: pd.Series, all_bigrams_df2: set) -> int:
        return int(any(bigram in all_bigrams_df2 for bigram in row1))

    def find_similar_words(self, word: str, word_list: List[str]) -> List[str]:
        return [w for w in word_list if fuzz.ratio(word, w) >= self.fuzz_threshold]

    def match_bigrams(self, exact_match: bool = False, ngram_match: bool = False) -> pd.DataFrame:
        self.format_bigrams_as_tuples(self.df1, self.column_name_df1)
        self.format_bigrams_as_tuples(self.df2, self.column_name_df2)

        all_bigrams_df1 = set(bigram for sublist in self.df1[self.column_name_df1] for bigram in sublist)
        all_bigrams_df2 = set(bigram for sublist in self.df2[self.column_name_df2] for bigram in sublist)

        if exact_match:
            self.result_df['Bigrama Presente'] = self.df1[self.column_name_df1].apply(
                lambda row: int(any(bigram in all_bigrams_df2 for bigram in row))
            )
        else:
            self.result_df['Bigrama Presente'] = self.df1[self.column_name_df1].apply(
                lambda row: self.check_bigram_presence(row, all_bigrams_df2)
            )

        self.result_df['Bigramas Localizados'] = self.df1[self.column_name_df1].apply(
            lambda row: [bigram for bigram in row if self.check_bigram_presence([bigram], all_bigrams_df2)]
        )

        if ngram_match:
            all_ngrams_df2 = set(ngram for sublist in self.df2[self.column_name_df2] for ngram in sublist)
            self.result_df['Ngrama Presente'] = self.df1[self.column_name_df1].apply(
                lambda row: int(any(ngram in all_ngrams_df2 for ngram in row))
            )

        # Concatenar os DataFrames independentemente das dimensões
        self.result_df = pd.concat([self.result_df, self.df1, self.df2], axis=1)

        return self.result_df

    def find_similar_words_in_dataframe(self, target_word: str, target_column: str, df: pd.DataFrame) -> List[str]:
        all_words = set(df[target_column].sum())
        return self.find_similar_words(target_word, all_words)


In [49]:
# Exemplo de uso: Passando nomes das colunas dos bigramas e FUZZ_THRESHOLD
bigrama_matcher = BigramaMatcher(df, df3, column_name_df1='Bigramas Preprocessados', column_name_df2='Bigramas Preprocessados', fuzz_threshold=80)
result_df = bigrama_matcher.match_bigrams()

# Exibir o resultado
#print(result_df)

In [50]:
result_df

,Bigrama Presente,Bigramas Localizados,Oferta,Bigramas Preprocessados,Oferta,Bigramas Preprocessados
0,0,[],Oferta especial na compra de um produto,"[(oferta, especi), (especi, compra), (compra, ...",Super desconto na aquisição de qualquer item,"[(super, desconto), (desconto, aquisicao), (aq..."
1,0,[],Desconto imperdível em diversos itens,"[(desconto, imperdivel), (imperdivel, diverso)...",Oferta irresistível em uma variedade de produtos,"[(oferta, irresistivel), (irresistivel, varied..."
2,1,"[(promocao, exclusiva)]",Promoção exclusiva para clientes VIP,"[(promocao, exclusiva), (exclusiva, client), (...",Promoção exclusiva para membros VIP,"[(promocao, exclusiva), (exclusiva, membro), (..."
3,1,"[(desta, semana)]",Aproveite os preços baixos desta semana,"[(aproveit, preco), (preco, baixo), (baixo, de...",Não perca as grandes ofertas desta semana,"[(perca, grand), (grand, oferta), (oferta, des..."
